In [1]:
import pandas as pd

data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [2]:
len(documents)

1226258

In [3]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


### Data Preprocessing

In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### Lemmatize example

In [6]:
print(WordNetLemmatizer().lemmatize('went', pos='v'))

go


#### Stemmer Example

In [7]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [8]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [9]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


 tokenized and lemmatized document: 
['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']


In [10]:
processed_docs = documents['headline_text'].map(preprocess)

In [11]:
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

### Bag of words on the dataset

In [12]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [13]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [14]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [15]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(162, 1), (240, 1), (292, 1), (589, 1), (838, 1), (3570, 1), (3571, 1)]

In [16]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 162 ("govt") appears 1 time.
Word 240 ("group") appears 1 time.
Word 292 ("vote") appears 1 time.
Word 589 ("local") appears 1 time.
Word 838 ("want") appears 1 time.
Word 3570 ("compulsori") appears 1 time.
Word 3571 ("ratepay") appears 1 time.


### TF-IDF

In [17]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [18]:
corpus_tfidf = tfidf[bow_corpus]

In [19]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5842699484464488),
 (1, 0.38798859072167835),
 (2, 0.5008422243250992),
 (3, 0.5071987254965034)]


### Running LDA using Bag of Words

In [20]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [21]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.041*"polic" + 0.026*"death" + 0.026*"case" + 0.026*"charg" + 0.025*"court" + 0.021*"murder" + 0.017*"woman" + 0.017*"face" + 0.015*"alleg" + 0.013*"shoot"
Topic: 1 
Words: 0.047*"trump" + 0.025*"world" + 0.020*"open" + 0.016*"island" + 0.013*"win" + 0.013*"return" + 0.012*"lose" + 0.012*"street" + 0.011*"sydney" + 0.010*"beach"
Topic: 2 
Words: 0.040*"coronavirus" + 0.033*"victoria" + 0.025*"live" + 0.021*"nation" + 0.017*"restrict" + 0.014*"water" + 0.013*"life" + 0.011*"plan" + 0.011*"covid" + 0.010*"park"
Topic: 3 
Words: 0.040*"queensland" + 0.026*"sydney" + 0.021*"bushfir" + 0.020*"crash" + 0.019*"adelaid" + 0.018*"die" + 0.015*"perth" + 0.015*"final" + 0.014*"miss" + 0.011*"break"
Topic: 4 
Words: 0.034*"year" + 0.020*"famili" + 0.019*"canberra" + 0.018*"melbourn" + 0.018*"tasmania" + 0.015*"jail" + 0.015*"australian" + 0.013*"work" + 0.013*"high" + 0.012*"tasmanian"
Topic: 5 
Words: 0.030*"govern" + 0.020*"health" + 0.018*"state" + 0.015*"school" + 0.014*"help

Cool! Can you distinguish different topics using the words in each topic and their corresponding weights?

### Running LDA using TF-IDF

In [22]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [23]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.022*"coronavirus" + 0.017*"covid" + 0.010*"victoria" + 0.008*"weather" + 0.008*"bushfir" + 0.007*"andrew" + 0.007*"flood" + 0.007*"michael" + 0.006*"age" + 0.006*"pandem"
Topic: 1 Word: 0.015*"govern" + 0.009*"queensland" + 0.008*"coronavirus" + 0.007*"victorian" + 0.007*"feder" + 0.007*"financ" + 0.006*"budget" + 0.006*"peter" + 0.006*"elect" + 0.005*"restrict"
Topic: 2 Word: 0.015*"australia" + 0.009*"sport" + 0.009*"test" + 0.007*"cricket" + 0.007*"gold" + 0.007*"india" + 0.007*"game" + 0.006*"territori" + 0.005*"fiji" + 0.005*"tasmania"
Topic: 3 Word: 0.013*"border" + 0.009*"violenc" + 0.009*"juli" + 0.008*"alan" + 0.007*"domest" + 0.007*"inquest" + 0.006*"onlin" + 0.005*"polic" + 0.005*"fish" + 0.005*"coronavirus"
Topic: 4 Word: 0.016*"crash" + 0.011*"christma" + 0.010*"climat" + 0.008*"fatal" + 0.008*"david" + 0.007*"korea" + 0.007*"north" + 0.007*"die" + 0.007*"truck" + 0.007*"video"
Topic: 5 Word: 0.008*"turnbul" + 0.007*"biden" + 0.006*"septemb" + 0.006*"hong"

### Classification of the topics

### Performance evaluation by classifying sample document using LDA Bag of Words model

In [24]:
processed_docs[4310]

['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']

In [25]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.6418790221214294	 
Topic: 0.030*"govern" + 0.020*"health" + 0.018*"state" + 0.015*"school" + 0.014*"help" + 0.013*"indigen" + 0.012*"communiti" + 0.012*"fund" + 0.012*"fight" + 0.011*"speak"

Score: 0.14101597666740417	 
Topic: 0.034*"year" + 0.020*"famili" + 0.019*"canberra" + 0.018*"melbourn" + 0.018*"tasmania" + 0.015*"jail" + 0.015*"australian" + 0.013*"work" + 0.013*"high" + 0.012*"tasmanian"

Score: 0.1295207291841507	 
Topic: 0.042*"australia" + 0.035*"australian" + 0.033*"elect" + 0.025*"donald" + 0.023*"kill" + 0.020*"coast" + 0.017*"border" + 0.013*"protest" + 0.012*"gold" + 0.012*"attack"

Score: 0.012513394467532635	 
Topic: 0.040*"coronavirus" + 0.033*"victoria" + 0.025*"live" + 0.021*"nation" + 0.017*"restrict" + 0.014*"water" + 0.013*"life" + 0.011*"plan" + 0.011*"covid" + 0.010*"park"

Score: 0.012512884102761745	 
Topic: 0.025*"news" + 0.023*"hous" + 0.017*"brisban" + 0.017*"busi" + 0.017*"peopl" + 0.016*"farmer" + 0.015*"time" + 0.014*"royal" + 0.014*"market

Our test document has the highest probability to be part of the topic on the top.

### Performance evaluation by classifying sample document using LDA TF-IDF model

In [26]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.7461456060409546	 
Topic: 0.015*"govern" + 0.009*"queensland" + 0.008*"coronavirus" + 0.007*"victorian" + 0.007*"feder" + 0.007*"financ" + 0.006*"budget" + 0.006*"peter" + 0.006*"elect" + 0.005*"restrict"

Score: 0.1537887006998062	 
Topic: 0.011*"final" + 0.009*"scott" + 0.008*"leagu" + 0.008*"wednesday" + 0.007*"world" + 0.007*"open" + 0.006*"australian" + 0.006*"rugbi" + 0.005*"grand" + 0.005*"georg"

Score: 0.012511026114225388	 
Topic: 0.014*"countri" + 0.012*"health" + 0.010*"hour" + 0.009*"royal" + 0.009*"morrison" + 0.009*"commiss" + 0.008*"rural" + 0.008*"stori" + 0.007*"indigen" + 0.006*"mental"

Score: 0.012508395127952099	 
Topic: 0.008*"turnbul" + 0.007*"biden" + 0.006*"septemb" + 0.006*"hong" + 0.006*"kong" + 0.006*"footag" + 0.005*"paul" + 0.005*"protest" + 0.005*"toni" + 0.005*"malcolm"

Score: 0.012508243322372437	 
Topic: 0.013*"border" + 0.009*"violenc" + 0.009*"juli" + 0.008*"alan" + 0.007*"domest" + 0.007*"inquest" + 0.006*"onlin" + 0.005*"polic" + 0.005*

Our test document has the highest probability to be part of the topic on the top.

### Testing model on unseen document

In [27]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.3277435898780823	 Topic: 0.042*"australia" + 0.035*"australian" + 0.033*"elect" + 0.025*"donald" + 0.023*"kill"
Score: 0.2055833637714386	 Topic: 0.025*"news" + 0.023*"hous" + 0.017*"brisban" + 0.017*"busi" + 0.017*"peopl"
Score: 0.18335017561912537	 Topic: 0.034*"year" + 0.020*"famili" + 0.019*"canberra" + 0.018*"melbourn" + 0.018*"tasmania"
Score: 0.18326352536678314	 Topic: 0.041*"polic" + 0.026*"death" + 0.026*"case" + 0.026*"charg" + 0.025*"court"
Score: 0.01667826808989048	 Topic: 0.040*"covid" + 0.037*"coronavirus" + 0.035*"australia" + 0.031*"china" + 0.026*"test"
Score: 0.016676699742674828	 Topic: 0.047*"trump" + 0.025*"world" + 0.020*"open" + 0.016*"island" + 0.013*"win"
Score: 0.01667666807770729	 Topic: 0.030*"govern" + 0.020*"health" + 0.018*"state" + 0.015*"school" + 0.014*"help"
Score: 0.016676297411322594	 Topic: 0.022*"call" + 0.018*"rise" + 0.018*"victorian" + 0.015*"morrison" + 0.015*"scott"
Score: 0.016676098108291626	 Topic: 0.040*"coronavirus" + 0.033*"v